In [10]:
from google.colab import drive
import numpy as np
from math import sqrt
import random
import networkx as nx
from scipy.spatial import distance
from scipy.optimize import linear_sum_assignment
import matplotlib.pyplot as plt

In [ ]:
drive.mount('/content/drive')

## **Reading data from TSPLib files**

In [12]:
class Line:
    def __init__(self, lines):
        self.lines = iter(lines)
        self.current = next(self.lines)
    def __next__(self):
        self.current = next(self.lines)
        return self.current
    def __iter__(self):
        return self

In [13]:
class LoadSample:

    props = ["DIMENSION"]

    def __init__(self):
        self.coords = []
        self.weights = None
        self.lines = None

    def load(self, file):
        with open(file, 'r') as f:
            self.lines = Line(f.read().splitlines())
        self.select()

    def select(self):
        self.specification = {}
        while True:
            line = self.lines.current
            if ':' in line:
                self.specification.update(self.read_spec())
            elif line.startswith('NODE_COORD_SECTION'):
                next(self.lines)
                self.coords = self.read_coords()
            else:
                break
        del self.lines

    def read_spec(self):
        key, value = self.lines.current.split(':', 1)
        key, value = key.strip(), value.strip()
        value = int(value) if key in self.props else value
        next(self.lines)
        return {key: value}

    def read_coords(self):
        coords = []
        while True:
            try:
                _, x, y = self.lines.current.split()
                coords.append((float(x), float(y)))
            except ValueError:
                break

            try:
                next(self.lines)
            except StopIteration:
                break
        return coords

In [14]:
def plot_graph(G):
  pos = nx.spring_layout(G)
  nx.draw(
      G,
      pos, with_labels=True,
      font_weight='bold',
      node_color='lightblue',
      font_color='black',
      font_size=10,
      node_size=700,
      edge_color='gray',
      width=1
  )

  nx.draw_networkx_edges(G, pos, edgelist=G.edges(), edge_color='blue', width=2)
  plt.show()

## **Loading coordinates or weights to memory**

In [15]:
def euclidean_distance(p1, p2):
    return distance.euclidean(p1, p2)

In [16]:
def graph_with_coords(loader):

  dim = loader.specification['DIMENSION']
  weights = np.full((dim, dim), -1)
  np.fill_diagonal(weights, 0)

  G = nx.Graph()

  n = 0
  for coord in loader.coords:
    G.add_node(n, pos=coord)
    n += 1

  for i in range(0, dim):
    for j in range(0, dim):
      weights[i][j] = euclidean_distance(G.nodes[i]['pos'], G.nodes[j]['pos'])
      G.add_edge(i, j, weight=weights[i][j])

  return (G, weights)

In [17]:
def graph_with_weights(loader):

  dim = loader.specification['DIMENSION']
  weights = np.array(loader.weights)
  weights = weights + weights.T - np.diag(weights.diagonal())

  G = nx.Graph()
  G.add_nodes_from(range(0, dim))

  for i in range(0, dim):
    for j in range(0, dim):
      G.add_edge(i, j, weight=weights[i][j])

  return (G, weights)

## **Approximative Travelling Salesman Problem Tour**

In [18]:
def dfs(adj, start_node):
    stack = [start_node]
    visited = set()
    H = []

    while stack:
        node = stack.pop()
        if node not in visited:
            visited.add(node)
            H.append(node)
            stack.extend(neighbor for neighbor in adj[node] if neighbor not in visited)

    return H

In [19]:
def hamiltonian_cycle_with_weights(H, weights):

  path = 0
  for i in range(len(H) - 1):
    j = i + 1
    path += weights[H[i]][H[j]]
  path += weights[H[-1], H[0]]
  return path


In [20]:
def approx_tsp_tour(loader, is_gr):

  G = None
  weights = None
  if is_gr: G, weights = graph_with_weights(loader)   # Loading graph
  else: G, weights = graph_with_coords(loader)

  G = nx.minimum_spanning_tree(G)                     # MST
  adj_list = nx.to_dict_of_lists(G)                   # MST adjacency list

  H = dfs(adj_list, 0)                                # Get hamiltonian cycle

  length = hamiltonian_cycle_with_weights(H, weights) # Traverse HC
  return int(length)

In [21]:
def call_tour(path):

  loader = LoadSample()
  loader.load(path)

  val = None
  if 'gr17' in path or 'gr21' in path:
    val = approx_tsp_tour(loader, True)
  else:
    val = approx_tsp_tour(loader, False)
  return val

## **Christofides Algorithm**

In [ ]:
class UnionFind:
    def __init__(self):
        self.weights = {}
        self.ancestors = {}

    def union(self, *nodes):
        roots = [self[x] for x in nodes]
        max_weighted = max([(self.weights[r], r) for r in roots])[1]
        for r in roots:
            if r != max_weighted:
                self.weights[max_weighted] += self.weights[r]
                self.ancestors[r] = max_weighted

    def __iter__(self):
        return iter(self.ancestors)

    def __getitem__(self, node):
        if node not in self.ancestors:
            self.ancestors[node] = node
            self.weights[node] = 1
            return node
        path = [node]
        root = self.ancestors[node]
        while root != path[-1]:
            path.append(root)
            root = self.ancestors[root]
        for ancestor in path:
            self.ancestors[ancestor] = root
        return root

In [23]:
def fill_weights(data):
    graph = {}
    for p1 in range(len(data)):
        for p2 in range(len(data)):
            if p1 != p2:
                if p1 not in graph:
                    graph[p1] = {}
                graph[p1][p2] = euclidean_distance(data[p1], data[p2])
    return graph

In [24]:
def minimum_spanning_tree(G):
    tree = []
    subtrees = UnionFind()
    for W, u, v in sorted((G[u][v], u, v) for u in G for v in G[u]):
        if subtrees[u] != subtrees[v]:
            tree.append((u, v, W))
            subtrees.union(u, v)
    return tree

In [25]:
def odd_subgraph(T):
    tmp_g = {}
    I = []
    for edge in T:
        if edge[0] not in tmp_g:
            tmp_g[edge[0]] = 0

        if edge[1] not in tmp_g:
            tmp_g[edge[1]] = 0

        tmp_g[edge[0]] += 1
        tmp_g[edge[1]] += 1

    for vertex in tmp_g:
        if tmp_g[vertex] % 2 == 1:
            I.append(vertex)

    return I

In [26]:
def minimum_weight_matching(T, G, odds):
    random.shuffle(odds)
    while odds:
        length = float("inf")
        u = 1
        v = odds.pop()
        closest = 0
        for u in odds:
            if v != u and G[v][u] < length:
                length = G[v][u]
                closest = u
        T.append((v, closest, length))
        odds.remove(closest)

In [27]:
def take_edge_out(T, v1, v2):
    for i, node in enumerate(T):
        if (node[0] == v2 and node[1] == v1) or (node[0] == v1 and node[1] == v2):
            del T[i] # delete edge
    return T

In [28]:
def create_neighbours_dict(G):
    neighbours = {}
    for edge in G:
        if edge[0] not in neighbours:
            neighbours[edge[0]] = []
        if edge[1] not in neighbours:
            neighbours[edge[1]] = []
        neighbours[edge[0]].append(edge[1])
        neighbours[edge[1]].append(edge[0])
    return neighbours

In [29]:
def eulerian_path(T, G):

    root = T[0][0] # root node
    neighbours = create_neighbours_dict(T) # create neighbours structure
    path = [neighbours[root][0]] # add root neighbours to path

    while len(T) > 0:
        for i, v in enumerate(path):
            if len(neighbours[v]) > 0:
                break
        while len(neighbours[v]) > 0:
            w = neighbours[v][0]
            take_edge_out(T, v, w)
            del neighbours[w][(neighbours[w].index(v))]
            del neighbours[v][(neighbours[v].index(w))]
            i += 1
            path.insert(i, w)
            v = w

    return path

In [30]:
def traverse_graph(G, path):
    current = path[0]
    visited = set()
    visited.add(current)
    length = 0
    for v in path:
        if v not in visited:
            visited.add(v)
            length += G[current][v]
            current = v

    length += G[current][path[0]]
    return length

In [31]:
# Eu queria ter feito usando NetworkX porém alguma
# coisa na hora de fazer o match perfeito mínimo
# estava dando errado, e o grafo não possuia um ciclo
# Euleriano no final : (

def christofides(nodes):

    G = fill_weights(nodes)               # Get distances
    T = minimum_spanning_tree(G)          # MST
    I = odd_subgraph(T)                   # I subgraph
    minimum_weight_matching(T, G, I)      # Minimum Perfect Match
    path = eulerian_path(T, G)            # Euler rolezinho

    length = traverse_graph(G, path)      # Twice Around
    return int(length)

In [32]:
def call_chris(path):

  loader = LoadSample()
  loader.load(path)

  if 'gr17' not in path or 'gr21' not in path:
    return christofides(loader.coords)


### Paths declaration

In [33]:
a280 = "/content/drive/MyDrive/Alg II/a280.tsp"
berlin52 = "/content/drive/MyDrive/Alg II/berlin52.tsp"
bier127 = "/content/drive/MyDrive/Alg II/bier127.tsp"
brd14051 = "/content/drive/MyDrive/Alg II/brd14051.tsp"
ch130 = "/content/drive/MyDrive/Alg II/ch130.tsp"
ch150 = "/content/drive/MyDrive/Alg II/ch150.tsp"
d1291 = "/content/drive/MyDrive/Alg II/d1291.tsp"
d15112 = "/content/drive/MyDrive/Alg II/d15112.tsp"
d1655 = "/content/drive/MyDrive/Alg II/d1655.tsp"
d18512 = "/content/drive/MyDrive/Alg II/d18512.tsp"
d198 = "/content/drive/MyDrive/Alg II/d198.tsp"
d2103 = "/content/drive/MyDrive/Alg II/d2103.tsp"
d493 = "/content/drive/MyDrive/Alg II/d493.tsp"
d657 = "/content/drive/MyDrive/Alg II/d657.tsp"
eil101 = "/content/drive/MyDrive/Alg II/eil101.tsp"
eil51 = "/content/drive/MyDrive/Alg II/eil51.tsp"
eil76 = "/content/drive/MyDrive/Alg II/eil76.tsp"
fl1400 = "/content/drive/MyDrive/Alg II/fl1400.tsp"
fl1577 = "/content/drive/MyDrive/Alg II/fl1577.tsp"
fl3795 = "/content/drive/MyDrive/Alg II/fl3795.tsp"
fl417 = "/content/drive/MyDrive/Alg II/fl417.tsp"
fnl4461 = "/content/drive/MyDrive/Alg II/fnl4461.tsp"
gil262 = "/content/drive/MyDrive/Alg II/gil262.tsp"
kroA100 = "/content/drive/MyDrive/Alg II/kroA100.tsp"
kroA150 = "/content/drive/MyDrive/Alg II/kroA150.tsp"
kroA200 = "/content/drive/MyDrive/Alg II/kroA200.tsp"
kroB100 = "/content/drive/MyDrive/Alg II/kroB100.tsp"
kroB150 = "/content/drive/MyDrive/Alg II/kroB150.tsp"
kroB200 = "/content/drive/MyDrive/Alg II/kroB200.tsp"
kroC100 = "/content/drive/MyDrive/Alg II/kroC100.tsp"
kroD100 = "/content/drive/MyDrive/Alg II/kroD100.tsp"
kroE100 = "/content/drive/MyDrive/Alg II/kroE100.tsp"
lin105 = "/content/drive/MyDrive/Alg II/lin105.tsp"
lin318 = "/content/drive/MyDrive/Alg II/lin318.tsp"
linhp318 = "/content/drive/MyDrive/Alg II/linhp318.tsp"
nrw1379 = "/content/drive/MyDrive/Alg II/nrw1379.tsp"
p654 = "/content/drive/MyDrive/Alg II/p654.tsp"
pcb1173 = "/content/drive/MyDrive/Alg II/pcb1173.tsp"
pcb3038 = "/content/drive/MyDrive/Alg II/pcb3038.tsp"
pcb442 = "/content/drive/MyDrive/Alg II/pcb442.tsp"
pr1002 = "/content/drive/MyDrive/Alg II/pr1002.tsp"
pr107 = "/content/drive/MyDrive/Alg II/pr107.tsp"
pr124 = "/content/drive/MyDrive/Alg II/pr124.tsp"
pr136 = "/content/drive/MyDrive/Alg II/pr136.tsp"
pr144 = "/content/drive/MyDrive/Alg II/pr144.tsp"
pr152 = "/content/drive/MyDrive/Alg II/pr152.tsp"
pr226 = "/content/drive/MyDrive/Alg II/pr226.tsp"
pr2392 = "/content/drive/MyDrive/Alg II/pr2392.tsp"
pr264 = "/content/drive/MyDrive/Alg II/pr264.tsp"
pr299 = "/content/drive/MyDrive/Alg II/pr299.tsp"
pr439 = "/content/drive/MyDrive/Alg II/pr439.tsp"
pr76 = "/content/drive/MyDrive/Alg II/pr76.tsp"
rat195 = "/content/drive/MyDrive/Alg II/rat195.tsp"
rat575 = "/content/drive/MyDrive/Alg II/rat575.tsp"
rat783 = "/content/drive/MyDrive/Alg II/rat783.tsp"
rat99 = "/content/drive/MyDrive/Alg II/rat99.tsp"
rd100 = "/content/drive/MyDrive/Alg II/rd100.tsp"
rd400 = "/content/drive/MyDrive/Alg II/rd400.tsp"
rl11849 = "/content/drive/MyDrive/Alg II/rl11849.tsp"
rl1304 = "/content/drive/MyDrive/Alg II/rl1304.tsp"
rl1323 = "/content/drive/MyDrive/Alg II/rl1323.tsp"
rl1889 = "/content/drive/MyDrive/Alg II/rl1889.tsp"
rl5915 = "/content/drive/MyDrive/Alg II/rl5915.tsp"
rl5934 = "/content/drive/MyDrive/Alg II/rl5934.tsp"
st70 = "/content/drive/MyDrive/Alg II/st70.tsp"
ts225 = "/content/drive/MyDrive/Alg II/ts225.tsp"
tsp225 = "/content/drive/MyDrive/Alg II/tsp225.tsp"
u1060 = "/content/drive/MyDrive/Alg II/u1060.tsp"
u1432 = "/content/drive/MyDrive/Alg II/u1432.tsp"
u159 = "/content/drive/MyDrive/Alg II/u159.tsp"
u1817 = "/content/drive/MyDrive/Alg II/u1817.tsp"
u2152 = "/content/drive/MyDrive/Alg II/u2152.tsp"
u2319 = "/content/drive/MyDrive/Alg II/u2319.tsp"
u574 = "/content/drive/MyDrive/Alg II/u574.tsp"
u724 = "/content/drive/MyDrive/Alg II/u724.tsp"
usa13509 = "/content/drive/MyDrive/Alg II/usa13509.tsp"
vm1084 = "/content/drive/MyDrive/Alg II/vm1084.tsp"
vm1748 = "/content/drive/MyDrive/Alg II/vm1748.tsp"

In [34]:
tsp_files = [
             (eil51, 426),
             (berlin52, 7542),
             (st70, 675),
             (eil76, 538),
             (pr76, 108159),
             (rat99, 1211),
             (kroA100, 21282),
             (kroB100, 22141),
             (kroC100, 20749),
             (kroD100, 21294),
             (kroE100, 22068),
             (rd100, 7910),
             (eil101, 629),
             (lin105, 14379),
             (pr107, 44303),
             (pr124, 59030),
             (bier127, 118282),
             (ch130, 6110),
             (pr136, 96772),
             (pr144, 58537),
             (ch150, 6528),
             (kroA150, 26524),
             (kroB150, 26130),
             (pr152, 73682),
             (u159, 42080),
             (rat195, 2323),
             (d198, 15780),
             (kroA200, 29368),
             (kroB200, 29437),
             (ts225, 126643),
             (tsp225, 3919),
             (pr226, 80369),
             (gil262, 2378),
             (pr264, 49135),
             (a280, 2579),
             (pr299, 48191),
             (lin318, 42029),
             (rd400, 15281),
             (fl417, 11861),
             (pr439, 107217),
             (pcb442, 50778),
             (d493, 35002),
             (u574, 36905),
             (rat575, 6773),
             (p654, 34643),
             (d657, 48912),
             (u724, 41910),
             (rat783, 8806),
             (pr1002, 259045),
             (u1060, 224094),
             (vm1084, 239297),
             (pcb1173, 56892),
             (d1291, 50801),
             (rl1304, 252948),
             (rl1323, 270199),
             (nrw1379, 56638),
             (fl1400, 20127),
             (u1432, 152970),
             (fl1577, 22249),
             (d1655, 62128),
             (vm1748, 336556),
             (u1817, 57201),
             (rl1889, 316536),
             (d2103, 80450),
             (u2152, 64253),
             (u2319, 234256),
             (pr2392, 378032),
             (pcb3038, 137694),
             (fl3795, 28772),
             (fnl4461, 182566),
             (rl5915, 565530),
             (rl5934, 556045),
             (rl11849, 923368),
             (usa13509, 19982889),
             (brd14051, 469445),
             (d15112, 1573152),
             (d18512, 645488)
             ]

## **Asserting Results**


In [ ]:
# Approximate Tour
for path, opt in tsp_files:

  print(path)
  length = call_tour(path)

  status = ""
  if length / 2 <= opt:
    status = "worked as planned"
  else:
    status = "did not worked as planned"

  output = f"FOR:  {path}   :   {length}\nThus the Approx Tour algorithm {status}\n"
  # with open("/content/drive/MyDrive/Alg II/results.txt", 'a') as file:
  #   file.write(output + '\n')

In [ ]:
# Christofides
for path, opt in tsp_files:

  print(path)
  length = call_chris(path)

  status = ""
  if length / 1.5 <= opt:
    status = "worked as planned"
  else:
    status = "did not worked as planned"

  output = f"FOR:  {path}   :   {length}\nThus the Christofides algorithm {status}\n"
  # with open("/content/drive/MyDrive/Alg II/results.txt", 'a') as file:
  #   file.write(output + '\n')